In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data
import streamlit as st
df = pd.read_csv("zRankings.csv")

In [2]:
state_map = data.us_10m.url

In [3]:
states = alt.topo_feature(state_map, feature='states')
us = alt.Chart(states).mark_geoshape(
    fill = '#f6f4e9',
    stroke='black',
    strokeWidth=0.5
)

In [4]:
pd.set_option('display.max_columns', 35)
df.head()


,Unnamed: 0,Zone,Resort Name,State,Base Elevation (ft),Top Elevation (ft),Snowfall Score,Total Snow Score With Preservation,Overall Ranking,Regional Ranking,State Ranking,True Snow Per Year (inches),Snow Quality Rank,Dump Potential Rank,MonthsMoreThan90Snow,MonthsLessThan30Snow,latitude,longitude,Acreage,Total Runs,Longest Run (ft),Lifts,Uphill Max (ppl/hr),Terrain Parks,Halfpipes
0,0,Rockies,Alta,UT,8530,10550,96.1,100.0,4,4,2,517,1.0,2.0,49.3,1.3,40.589613,-111.638825,2200,50,16800,11,11248,0,0
1,1,Rockies,Snowbird,UT,7760,11000,90.0,97.5,3,3,1,497,2.0,5.0,37.0,3.4,40.583257,-111.655183,2500,85,13200,11,17400,1,0
2,2,Rockies,Brighton,UT,8750,10500,92.4,95.2,20,16,5,504,3.0,3.0,44.2,4.3,40.598672,-111.583739,1050,66,15840,6,10100,4,2
3,3,Canadian-Rockies,Powder King,BC,3000,5500,92.3,93.3,51,6,6,492,4.0,4.0,45.0,3.0,55.364552,-122.619405,925,37,2800,3,2600,0,0
4,4,Rockies,Grand Targhee,WY,7408,10000,88.9,91.9,19,15,2,466,5.0,6.0,40.4,3.5,43.787501,-110.959235,3000,72,14675,5,7200,1,0


In [5]:
points = alt.Chart(df).mark_circle(opacity=0.9, stroke = 'white').encode(
    latitude='latitude:Q',
    longitude='longitude:Q',
    size=alt.Size('True Snow Per Year (inches):Q', scale=alt.Scale(range=[0,660]), legend = alt.Legend(title = 'True Snow Per Year (inches)')),
    color=alt.Color('Snowfall Score:Q', scale=alt.Scale(domain=[30, 50, 70, 90], scheme='blueorange'), legend=alt.Legend(title='Snowfall Score')),
    tooltip = [
        alt.Tooltip('Resort Name', title='Resort Name'),
        alt.Tooltip('True Snow Per Year (inches):Q', title='Snow Per Year (inches)'),
        alt.Tooltip('MonthsMoreThan90Snow', title='Extended Stay'),
        alt.Tooltip('MonthsLessThan30Snow:Q', title='Drought Threat')
        ])

In [6]:
import geopandas as gpd

fp = gpd.datasets.get_path('naturalearth_lowres')
gdf_ne = gpd.read_file(fp)
gdf_sel = gdf_ne[gdf_ne.continent == 'North America']
namap = alt.Chart(gdf_sel).mark_geoshape(fill='lightgray',
    stroke='white',
    strokeWidth=0.5)

In [7]:
chart = alt.layer(
    namap,us, points).properties(
    width=800,
    height=500).project(
    scale = 430,
    translate=[1260, 650]) 

In [8]:
title = alt.TitleParams(
    'Snowfall at Ski Resorts in North America',
    subtitle='Data source: ZRankings',
    subtitleColor='#555',
    dy=-20,
    fontSize=20
)
chart.configure_title(
    fontSize=24,
    anchor='middle',
    color='#333'
).configure_legend(
    labelFontSize=12,
    titleFontSize=12,
    padding=30
).configure_view(
    strokeWidth=0
).properties(
    title=title
)


alt.LayerChart(...)

In [9]:
brush = alt.selection_interval(encodings=['x'])

histogram = alt.Chart(df).mark_bar().encode(
    x=alt.X('Snowfall Score:Q', bin=alt.Bin(step=10), axis=alt.Axis(title='Snowfall Score')),
    y=alt.Y('count()', axis=alt.Axis(title='Number of Resorts')),
    color=alt.condition(brush, alt.value('#e39e19'), alt.value('lightgray'))
).properties(
    width=700,
    height=150
).add_selection(
    brush
)

resort_count = alt.Chart(df).transform_filter(
    brush
).mark_text(
    fontSize=48,
    fontWeight=600,
    color='#e39e19'
).encode(
    text='count()'
)

histogram & resort_count

alt.VConcatChart(...)